# Image Generator

**사용방법**
* 사용환경 : google colab 서버 사용_런타임 연결 유지를 위해 colab pro와 T4 GPU 사용, ngrok 서버 운영, local python inputs
* 먼저 ngrok.exe를 실행 후, **ngrok http 7860**를 입력 -> ngrok 서버 작동 시작 (1번 실행 후 창 내려놓으면 됨)
* 이후 colab의 Colab_Webui_GPU_server.ipynb 실행 후 출력창의 맨 밑에 나오는 ngrok 링크 복사 (1번 실행)
* 복사한 링크를 Local_prompt_inputs.py의 ===================  해당 부분에 넣기!! ==============
    url="https://3c19a4b14804.ngrok.app/sdapi/v1/txt2img"
    라고 적힌 부분에서 https~app 부분에 붙여넣기 (1회)

*이제 프롬프트만 수정하면 3분내로 이미지 계속 출력 가능    

**개선점**
* 이미지가 한개만 출력됨 -> 일단 출력시간을 고려해서 하나만 출력하긴 했으나 이걸 어떻게 빠른 시간에 일관적인 이미지를 많이 뽑느냐의 문제가 존재. 표정,행동 등

* 출력시간이 3분정도 걸리는것 1분 이내로 줄이기 -> 네트워크 or GPU

* 배경은 아직 이뤄지지 않았는데 사람 말고 배경 이미지도 입력이 될련지

Colab_Webui_GPU_server.ipynb

In [ ]:
import os
import shutil
import subprocess
import threading
import time
import requests

# ✅ 기본 경로 설정
base_path = "/content/stable-diffusion-webui"
vae_path = os.path.join(base_path, "models", "VAE", "vae-ft-mse-840000-ema-pruned.safetensors")

# ✅ 0. 기존 설정 파일 초기화 (기본 모델 무시 방지)
config_files = [
    os.path.join(base_path, "config.json"),
    os.path.join(base_path, "ui-config.json"),
    os.path.join(base_path, "extensions", "sd-webui-controlnet", "config.json")
]
for config_file in config_files:
    if os.path.exists(config_file):
        print(f"🧹 설정 파일 제거: {config_file}")
        os.remove(config_file)

# ✅ WebUI 폴더 없으면 설치 진행
if not os.path.exists(base_path):
    !rm -rf {base_path}
    %cd /content

    # ✅ 1. 필수 패키지 설치
    !pip install -q --upgrade pip
    !pip uninstall -y xformers lightning_fabric wandb pytorch_lightning
    !pip install -q \
      pytorch_lightning==1.9.5 \
      gradio==3.41.2 \
      pydantic==1.10.13 \
      open_clip_torch==2.24.0 \
      omegaconf==2.3.0 \
      jsonmerge clean-fid resize-right tomesd blendmodes lark \
      opencv-python insightface diffusers accelerate transformers datasets scipy \
      kornia piexif pillow-avif-plugin diskcache facexlib==0.3.0 inflection==0.5.1

    !pip install git+https://github.com/google-research/torchsde.git
    !pip install git+https://github.com/rtqichen/torchdiffeq
    !pip install git+https://github.com/openai/CLIP.git
    !pip install git+https://github.com/patrickvonplaten/controlnet_aux.git
    !pip install spandrel

    # ✅ 2. WebUI 및 모델 다운로드
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui {base_path}
    !mkdir -p {base_path}/models/Stable-diffusion
    !mkdir -p {base_path}/models/Lora

    model_urls = {
        "AnyLoRA_v1.0.safetensors": "https://huggingface.co/PODOISTHEBEST/CharacterCreaterforStory/resolve/main/AnyLoRA_v1.0.safetensors",
        "ruirui_ill_v1.0.safetensors": "https://huggingface.co/PODOISTHEBEST/CharacterCreaterforStory/resolve/main/ruirui_ill_v1.0.safetensors",
        "pastelMixStylizedAnime_pastelMixLoraVersion.safetensors": "https://huggingface.co/PODOISTHEBEST/CharacterCreaterforStory/resolve/main/pastelMixStylizedAnime_pastelMixLoraVersion.safetensors"
    }

    for name, url in model_urls.items():
        folder = "Lora" if "lora" in name.lower() else "Stable-diffusion"
        path = f"{base_path}/models/{folder}/{name}"
        if not os.path.exists(path):
            print(f"🗕 {name} 다운로드 중...")
            os.system(f"wget -O {path} {url}")
        else:
            print(f"✅ {name} 이미 있음")

    # ✅ ControlNet + GFPGAN
    !git clone https://github.com/Mikubill/sd-webui-controlnet {base_path}/extensions/sd-webui-controlnet
    !git clone https://github.com/TencentARC/GFPGAN.git {base_path}/repositories/GFPGAN
    %cd {base_path}/repositories/GFPGAN
    !pip install -r requirements.txt

# ✅ 3. VAE 파일이 없으면 다운로드
if not os.path.exists(vae_path):
    print("🔻 VAE 모델 없음 → 다운로드 시작")
    os.makedirs(os.path.dirname(vae_path), exist_ok=True)
    !wget -O {vae_path} https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors
else:
    print("✅ VAE 모델 이미 존재")

# ✅ 4. AnyLoRA를 기본 모델로 이동 및 이름 변경 (우선순위 확보)
original_name = "AnyLoRA_v1.0.safetensors"
priority_name = "0_AnyLoRA_v1.0.safetensors"
lora_path = os.path.join(base_path, "models", "Lora", original_name)
default_model_path = os.path.join(base_path, "models", "Stable-diffusion", priority_name)

# Lora → Stable-diffusion 이동 및 이름 변경
if os.path.exists(lora_path):
    print("🔁 AnyLoRA 모델 이동 및 이름 변경")
    shutil.move(lora_path, default_model_path)
elif not os.path.exists(default_model_path):
    print("❌ AnyLoRA 모델 경로를 찾을 수 없습니다.")

# ✅ 5. WebUI 실행 (기본 모델 명시)
webui_args = [
    "python", "launch.py", "--lowvram", "--listen", "--port", "5000", "--api",
    "--disable-safe-unpickle", "--disable-nan-check", "--no-half", "--precision", "full",
    "--no-download-sd-model", "--disable-opt-split-attention", "--skip-install"
]

if os.path.exists(default_model_path):
    print(f"✅ 기본 모델로 AnyLoRA 지정: {default_model_path}")
    webui_args += ["--ckpt", default_model_path]
else:
    print("⚠️ AnyLoRA 모델 없음 → 기본 지정 생략")

webui_proc = subprocess.Popen(
    webui_args,
    cwd=base_path,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# ✅ 로그 스트리밍
def stream_logs():
    with open("/content/webui_log.txt", "w") as f:
        for line in iter(webui_proc.stdout.readline, ""):
            print(line.strip())
            f.write(line)
            f.flush()

threading.Thread(target=stream_logs, daemon=True).start()

# ✅ 로컬 접속 확인
for _ in range(30):
    try:
        if requests.get("http://127.0.0.1:5000").status_code == 200:
            print("✅ WebUI 로컬 접속 성공")
            break
    except:
        pass
    time.sleep(2)

# ✅ Ngrok 연결
!wget -O ngrok-v3.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3.tgz && mv ngrok /usr/local/bin && chmod +x /usr/local/bin/ngrok
!ngrok config add-authtoken 2oT1YaHvZh3gDgiESvtLQmPcwtw_38dvMEDJjP6aHYcobSFZ
!nohup /usr/local/bin/ngrok http 5000 &> /dev/null &

def get_ngrok_url():
    try:
        res = requests.get("http://localhost:4040/api/tunnels").json()
        return res['tunnels'][0]['public_url']
    except Exception as e:
        print("❌ ngrok URL 오류:", e)
        return None

time.sleep(8)
public_url = get_ngrok_url()
if public_url:
    print(f"✅ Ngrok 외부 URL: {public_url}")
else:
    print("❌ Ngrok 실패")


/content
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.2 MB/s eta 0:00:00
Found existing installation: wandb 0.19.11
Uninstalling wandb-0.19.11:
  Successfully uninstalled wandb-0.19.11
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 168.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 177.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 118.5 MB/s eta 0:00:

Local_prompt_inputs.py

In [ ]:
import requests
import base64
from PIL import Image
from io import BytesIO
import pprint
import os

# 📌 기본 프롬프트 (아이 대상 캐릭터 생성)
BASE_PROMPT = (
    "a wholesome, child-safe, kindergarten-aged cartoon character, "
    "in a long-sleeved pastel clothes, wearing tights and shoes, "
    "friendly and cute, colorful fairytale style, full body, solo, 1girl, 1boy "
    "Studio Ghibli style, White background"
)

def select_behavior_prompt():
    options = {
        "1": "sleeping peacefully, eyes closed, lying on side, arms tucked under head, gentle smile",
        "2": "happy face, smile, grin ,laugh",
        "3": "running forward with energy, arms swinging, excited expression, one foot off the ground, fast motion",
        "4": "frown , angry face, shouting",
        "5": "sitting on a small chair, legs dangling, relaxed pose, happy face, hands resting on lap",
        "6": "jumping high, arms up, big smile, hair bouncing, energetic motion, floating in the air",
        "7": "waving hand with a smile, one arm raised, bright eyes, friendly expression, standing posture",
        "8": "(crying:0.1), sad, tearful face , cry out, blush",
        "9": "(shocked expression:0.1), surprised face ,mouth open, hands on cheeks, slightly leaning back, wide eyes",
        "":  "standing still, neutral expression, looking straight ahead"  # 기본 자세
    }

    print("\n🧍 행동/표정 선택 (엔터 누르면 기본자세):")
    for key in options:
        if key:
            print(f"{key}. {options[key].split(',')[0].capitalize()}")

    choice = input("번호를 선택하세요 (1~9): ").strip()
    base_behavior = options.get(choice, options[""])

    custom_behavior = input("✏️ 추가 행동/표정 프롬프트를 입력하세요 (없으면 엔터):\n> ").strip()

    return f"{base_behavior}, {custom_behavior}" if custom_behavior else base_behavior


def build_full_prompt():
    appearance = input("🎨 캐릭터 외형을 묘사하세요 (ex. pink magical dress, long hair, tiara):\n> ").strip()
    behavior = select_behavior_prompt()
    full_prompt = f"{BASE_PROMPT}, {appearance}, {behavior}"
    print(f"\n🧠 최종 프롬프트:\n{full_prompt}\n")
    return full_prompt


def generate_image_advanced(
    prompt,
    negative_prompt=(
        "naked, lingerie, cleavage, breasts, exposed skin, "
        "revealing outfit, tight clothes, suggestive pose, [sexualized:0.2], erotic, lewd, "
        "short skirt, open shirt, inappropriate, unrealistic proportions, "
        "bad anatomy, poorly drawn hands, extra limbs, blurry, low quality, scary, creepy, dark shadows"
    ),
    save_path="generated_result.png",
    lora_name=None,
    controlnet_input_image_path=None,
    controlnet_module="canny",
    controlnet_model="control_v11p_sd15_canny [d14c016b]",
    upscale=False,
    disable_refiner=True,
    url="https://4829a66bfe41.ngrok.app/sdapi/v1/txt2img"
):
    full_prompt = f"<lora:{lora_name}:1.0> {prompt}" if lora_name else prompt

    payload = {
        "prompt": full_prompt,
        "negative_prompt": negative_prompt,
        "steps": 30,
        "sampler_name": "DPM++ 2M",
        "scheduler": "Karras",
        "cfg_scale": 7,
        "width": 280,
        "height": 410,
        "seed": 2873609975,
        "enable_hr": bool(upscale),
        "hr_upscaler": "Latent",
        "hr_scale": 1.5,
        "hr_second_pass_steps": 15,
        "alwayson_scripts": {},
        "sd_model_checkpoint": "myModel.ckpt",
        "override_settings": {"CLIP_stop_at_last_layers": 2}
    }

    if disable_refiner:
        payload["refiner_checkpoint"] = None
        payload["refiner_switch_at"] = None

    if controlnet_input_image_path and os.path.exists(controlnet_input_image_path):
        with open(controlnet_input_image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode()

        payload["alwayson_scripts"]["controlnet"] = {
            "args": [
                {
                    "enabled": True,
                    "input_image": encoded_image,
                    "module": controlnet_module,
                    "model": controlnet_model,
                    "control_mode": 0,
                    "weight": 0.6,
                    "resize_mode": 1,
                    "guidance": 1,
                    "processor_res": 512,
                    "threshold_a": 50,
                    "threshold_b": 150,
                    "starting_control_step": 0.0,
                    "ending_control_step": 0.5
                }
            ]
        }

    payload = {k: v for k, v in payload.items() if v is not None}

    print("📤 요청 중...")
    pprint.pprint(payload)

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        result = response.json()
        print("🔁 응답 원문:", result)
    except Exception as e:
        print("❌ 응답 파싱 오류:", e)
        print("본문 내용:", getattr(e, 'response', None).text if hasattr(e, 'response') else 'N/A')
        return

    if "images" not in result:
        raise Exception(f"❌ 이미지 생성 실패: {result}")

    image_data = result["images"][0]
    if "," in image_data:
        image_data = image_data.split(",", 1)[1]
    image = Image.open(BytesIO(base64.b64decode(image_data)))
    image.save(save_path)
    print(f"✅ 이미지 저장 완료: {save_path}")
    image.show()
    return image


if __name__ == "__main__":
    final_prompt = build_full_prompt()
    generate_image_advanced(
        prompt=final_prompt,
        lora_name=None,
        upscale=False,
        disable_refiner=True,
        save_path="child_friendly_character.png"
    )


🎨 캐릭터 외형을 묘사하세요 (ex. pink magical dress, long hair, tiara):
> Create a full-body character design of a cheerful squirrel mascot. It has soft, light brown fur, big round eyes with sparkles, and a fluffy tail. The squirrel wears a green vest with a little acorn badge and a red bow tie. It stands upright like a human, with small paws waving in the air. Its expression is friendly and curious, perfect for a children’s educational app.

🧍 행동/표정 선택 (엔터 누르면 기본자세):
1. Sleeping peacefully
2. Sitting at a table
3. Running forward with energy
4. Playfully rolling on the ground
5. Sitting on a small chair
6. Jumping high
7. Waving hand with a smile
8. Crying
9. Shocked expression
번호를 선택하세요 (1~9): 1
✏️ 추가 행동/표정 프롬프트를 입력하세요 (없으면 엔터):
> 

🧠 최종 프롬프트:
a wholesome, child-safe, kindergarten-aged cartoon character, in a long-sleeved pastel clothes, wearing tights and shoes, friendly and cute, colorful fairytale style, full body, Studio Ghibli style, White background, Create a full-body character design of 